In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + "5"

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
#df_info.to_excel('전체법.xlsx',index = False)

In [4]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
687,해당
688,포함
689,거나
690,하다


In [5]:
posts = df_info.get("조내용")
posts_key = df_info.get("키")
key = [i for i in posts_key]
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [6]:
def get_key(**key) :
    dic = {}
    for k, y in key.items():
        if k[-1] == "1" :
            s = "산업안전보건법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="2":
            s = "건설기술진흥법 제 "+k[0:(len(k)-4)]+"조"하지만 😴😭
        elif k[-1]=="3":
            s = "위험물안전관리법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="4":
            s = "전기안전관리법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="5":
            s = "화학물질관리법 제 "+k[0:(len(k)-4)]+"조"     
        if k[-3] != "0":
            s += "의 "+k[-3]   
        dic[s] = y
    return dic   

In [7]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [8]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return(o)

tokenizer_1(posts[80])


['81조',
 '기계',
 '기구',
 '여자',
 '조치',
 '기계',
 '기구',
 '설비',
 '건축물',
 '대여',
 '대여받다',
 '자다',
 '안전',
 '조치',
 '보건',
 '조치']

In [9]:
#from konlpy.tag import Mecab
#tagger = Mecab()
from eunjeon import Mecab
tagger = Mecab()
def tokenizer_2(post):
    nouns = []
  
    for noun in tagger.nouns(post):
        if noun not in list(stop_words_df.get("불용어")):
            nouns.append(noun)
    return nouns

tokenizer_2(posts[0])

['목적',
 '법',
 '안전',
 '보건',
 '기준',
 '확립',
 '책임',
 '소재',
 '재해',
 '예방',
 '작업',
 '환경',
 '조성',
 '노무',
 '제공',
 '사람',
 '안전',
 '보건',
 '유지',
 '목적',
 '개정']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [12]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,1.289757,1.414214,1.175499,1.248485,1.343371,1.361378,1.361695,1.309491,1.350362,...,1.378859,1.391069,1.398336,1.390360,1.380424,1.414214,1.397602,1.409579,1.414214,1.402701
1,1.289757,0.000000,1.327635,1.336567,1.213869,1.300910,1.341292,1.345794,1.355732,1.229740,...,1.387232,1.388835,1.396432,1.372453,1.384828,1.407918,1.383536,1.388266,1.397695,1.378485
2,1.414214,1.327635,0.000000,1.399259,1.365332,1.386404,1.414214,1.381233,1.414214,1.356870,...,1.379108,1.299542,1.400848,1.378431,1.395492,1.395183,1.395197,1.398395,1.414214,1.395249
3,1.175499,1.336567,1.399259,0.000000,1.300796,1.260531,1.345828,1.291897,1.267851,1.359846,...,1.280975,1.321514,1.395897,1.370300,1.360045,1.402729,1.379076,1.400056,1.392232,1.386225
4,1.248485,1.213869,1.365332,1.300796,0.000000,1.174273,1.365394,1.344439,1.331973,1.319417,...,1.391605,1.398829,1.393355,1.321130,1.336976,1.389226,1.336022,1.354460,1.401320,1.384685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.414214,1.407918,1.395183,1.402729,1.389226,1.404570,1.408413,1.400976,1.407867,1.378607,...,1.405327,1.377534,1.281977,1.289129,1.346569,0.000000,1.277802,1.309684,1.414214,1.363454
453,1.397602,1.383536,1.395197,1.379076,1.336022,1.369152,1.348739,1.375573,1.381954,1.368932,...,1.401072,1.391729,1.342874,0.977006,1.215813,1.277802,0.000000,1.060126,1.409455,1.074164
454,1.409579,1.388266,1.398395,1.400056,1.354460,1.407796,1.392560,1.388752,1.409989,1.392056,...,1.404550,1.389393,1.363714,1.119819,1.217507,1.309684,1.060126,0.000000,1.373568,1.228051
455,1.414214,1.397695,1.414214,1.392232,1.401320,1.394538,1.414214,1.389627,1.414214,1.414214,...,1.403486,1.394175,1.363075,1.376693,1.357506,1.414214,1.409455,1.373568,0.000000,1.391955


In [13]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
jo_key = 240011
jo = key.index(jo_key)
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:15]

[('산업안전보건법 제 24조', 0.0),
 ('산업안전보건법 제 75조', 0.9339417706976405),
 ('산업안전보건법 제 26조', 1.0332380575761178),
 ('산업안전보건법 제 25조', 1.1056151046058686),
 ('화학물질관리법 제 7조', 1.1500754790486534),
 ('산업안전보건법 제 49조', 1.1758099030796934),
 ('산업안전보건법 제 35조', 1.1796111549562123),
 ('산업안전보건법 제 44조', 1.205628661927156),
 ('건설기술진흥법 제 84조', 1.2072260458479738),
 ('산업안전보건법 제 116조', 1.2217613130534635),
 ('산업안전보건법 제 55조', 1.2279999869394227),
 ('산업안전보건법 제 18조', 1.2283545842192432),
 ('건설기술진흥법 제 5조', 1.2333638966861609),
 ('산업안전보건법 제 22조', 1.2343184588396674),
 ('산업안전보건법 제 13조', 1.242207774097933)]

In [14]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [15]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.168263,0.000000,0.309101,0.220643,0.097677,0.073324,0.072893,0.142616,0.088261,...,0.049374,0.032464,0.022328,0.033449,0.047215,0.000000,0.023354,0.006543,0.000000,0.016215
1,0.168263,1.000000,0.118693,0.106794,0.263261,0.153817,0.100468,0.094419,0.080995,0.243870,...,0.037794,0.035569,0.024989,0.058187,0.041125,0.008884,0.042914,0.036359,0.023224,0.049890
2,0.000000,0.118693,1.000000,0.021037,0.067934,0.038943,0.000000,0.046098,0.000000,0.079452,...,0.049031,0.155595,0.018813,0.049964,0.026300,0.026732,0.026713,0.022246,0.000000,0.026640
3,0.309101,0.106794,0.021037,1.000000,0.153965,0.205530,0.094374,0.165502,0.196276,0.075409,...,0.179552,0.126800,0.025736,0.061139,0.075139,0.016175,0.049075,0.019921,0.030845,0.039190
4,0.220643,0.263261,0.067934,0.153965,1.000000,0.310542,0.067849,0.096242,0.112925,0.129570,...,0.031717,0.021638,0.029281,0.127308,0.106247,0.035025,0.107522,0.082719,0.018151,0.041324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.008884,0.026732,0.016175,0.035025,0.013591,0.008186,0.018633,0.008955,0.049721,...,0.012528,0.051200,0.178267,0.169073,0.093376,1.000000,0.183611,0.142364,0.000000,0.070497
453,0.023354,0.042914,0.026713,0.049075,0.107522,0.062712,0.090451,0.053900,0.045102,0.063012,...,0.018498,0.031545,0.098345,0.522730,0.260899,0.183611,1.000000,0.438066,0.006718,0.423086
454,0.006543,0.036359,0.022246,0.019921,0.082719,0.009055,0.030388,0.035684,0.005965,0.031090,...,0.013620,0.034794,0.070142,0.373003,0.258838,0.142364,0.438066,1.000000,0.056655,0.245946
455,0.000000,0.023224,0.000000,0.030845,0.018151,0.027632,0.000000,0.034468,0.000000,0.000000,...,0.015114,0.028139,0.071014,0.052358,0.078588,0.000000,0.006718,0.056655,1.000000,0.031230


In [16]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('산업안전보건법 제 24조', 1.0000000000000002),
 ('산업안전보건법 제 75조', 0.5638763844730781),
 ('산업안전보건법 제 26조', 0.4662095581881659),
 ('산업안전보건법 제 25조', 0.38880762023367726),
 ('화학물질관리법 제 7조', 0.3386631962455054),
 ('산업안전보건법 제 49조', 0.3087355359098611),
 ('산업안전보건법 제 35조', 0.3042587615514354),
 ('산업안전보건법 제 44조', 0.27322976476986816),
 ('건설기술진흥법 제 84조', 0.2713026371131332),
 ('산업안전보건법 제 116조', 0.2536496469629385)]

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '못하다하기보다는', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent w

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [18]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [20]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,25.059928,8.717798,14.000000,11.180340,7.071068,9.433981,24.392622,12.083046,16.155494,...,8.366600,6.708204,6.480741,19.287302,22.203603,6.082763,13.601471,11.135529,10.000000,31.016125
1,25.059928,0.000000,24.372115,28.390139,24.145393,24.859606,26.210685,33.570821,27.166155,27.184554,...,26.038433,25.709920,25.690465,31.240999,33.211444,25.787594,28.337255,27.239677,26.381812,39.217343
2,8.717798,24.372115,0.000000,17.320508,12.529964,8.602325,10.908712,24.637370,13.638182,16.522712,...,9.165151,7.810250,8.124038,19.798990,22.781571,7.681146,14.456832,12.083046,11.224972,31.336879
3,14.000000,28.390139,17.320508,0.000000,16.462078,15.811388,17.578396,26.551836,17.832555,21.840330,...,16.852300,16.763055,16.613248,24.207437,26.210685,16.462078,20.223748,18.867962,18.275667,33.852622
4,11.180340,24.145393,12.529964,16.462078,0.000000,9.949874,13.711309,25.298221,14.866069,17.549929,...,12.922848,12.806248,12.369317,21.283797,23.664319,12.165525,16.552945,15.066519,14.594520,32.388269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,6.082763,25.787594,7.681146,16.462078,12.165525,6.855655,8.717798,24.289916,12.288206,15.684387,...,8.062258,5.291503,3.316625,18.627936,21.771541,0.000000,12.247449,9.643651,9.110434,30.446675
453,13.601471,28.337255,14.456832,20.223748,16.552945,13.820275,14.696938,26.944387,16.881943,19.646883,...,14.730920,13.490738,12.449900,14.456832,19.748418,12.247449,0.000000,9.746794,15.394804,23.558438
454,11.135529,27.239677,12.083046,18.867962,15.066519,11.661904,12.529964,25.903668,15.491933,18.303005,...,12.328828,10.816654,10.000000,15.684387,19.570386,9.643651,9.746794,0.000000,12.806248,27.495454
455,10.000000,26.381812,11.224972,18.275667,14.594520,10.488088,11.789826,25.553865,14.560220,17.748239,...,11.135529,9.433981,9.273618,19.899749,22.649503,9.110434,15.394804,12.806248,0.000000,31.464265


In [21]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:10]

[('산업안전보건법 제 24조', 0.0),
 ('산업안전보건법 제 25조', 18.76166303929372),
 ('산업안전보건법 제 49조', 18.76166303929372),
 ('산업안전보건법 제 32조', 19.026297590440446),
 ('산업안전보건법 제 35조', 19.8997487421324),
 ('산업안전보건법 제 19조', 20.12461179749811),
 ('산업안전보건법 제 26조', 20.12461179749811),
 ('산업안전보건법 제 4조', 20.396078054371138),
 ('산업안전보건법 제 20조', 20.8806130178211),
 ('산업안전보건법 제 31조', 21.166010488516726)]

In [22]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [23]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.203030,0.052870,0.567462,0.382255,0.221470,0.047378,0.063709,0.178279,0.061602,...,0.155700,0.043033,0.046676,0.051071,0.061489,0.000000,0.045083,0.019342,0.000000,0.043730
1,0.203030,1.000000,0.305926,0.136814,0.352476,0.237673,0.081763,0.090544,0.095426,0.200113,...,0.084298,0.061159,0.056860,0.035254,0.028535,0.000000,0.024409,0.023562,0.073171,0.041856
2,0.052870,0.305926,1.000000,0.075942,0.238729,0.180656,0.000000,0.085260,0.046271,0.105523,...,0.222260,0.184289,0.066630,0.058323,0.043887,0.086874,0.042904,0.041416,0.000000,0.044589
3,0.567462,0.136814,0.075942,1.000000,0.355876,0.262573,0.083176,0.183022,0.227625,0.062922,...,0.149097,0.027472,0.044697,0.055426,0.081312,0.038851,0.047968,0.024696,0.014380,0.067799
4,0.382255,0.352476,0.238729,0.355876,1.000000,0.571442,0.122245,0.154110,0.230000,0.217226,...,0.189710,0.018506,0.100362,0.105419,0.124657,0.104685,0.116325,0.074860,0.029059,0.061790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.000000,0.086874,0.038851,0.104685,0.000000,0.000000,0.026171,0.000000,0.080978,...,0.000000,0.070711,0.613572,0.151053,0.115470,1.000000,0.296319,0.254257,0.000000,0.164243
453,0.045083,0.024409,0.042904,0.047968,0.116325,0.064187,0.067283,0.035544,0.065761,0.054989,...,0.010529,0.017461,0.246205,0.642385,0.452649,0.296319,1.000000,0.659234,0.000000,0.707211
454,0.019342,0.023562,0.041416,0.024696,0.074860,0.000000,0.049485,0.024953,0.000000,0.025736,...,0.013552,0.022473,0.195006,0.554755,0.449560,0.254257,0.659234,1.000000,0.047052,0.476324
455,0.000000,0.073171,0.000000,0.014380,0.029059,0.000000,0.000000,0.009686,0.009857,0.007493,...,0.031564,0.052342,0.056773,0.099390,0.106843,0.000000,0.000000,0.047052,1.000000,0.060789


In [24]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('산업안전보건법 제 24조', 1.0),
 ('산업안전보건법 제 26조', 0.7979356967643391),
 ('산업안전보건법 제 35조', 0.7024085188547925),
 ('산업안전보건법 제 25조', 0.7014382462927293),
 ('산업안전보건법 제 49조', 0.690056685213648),
 ('산업안전보건법 제 32조', 0.6717577223590775),
 ('산업안전보건법 제 47조', 0.6416723324477737),
 ('산업안전보건법 제 19조', 0.637262128607455),
 ('산업안전보건법 제 20조', 0.6122281342184457),
 ('산업안전보건법 제 4조', 0.6078103237369795)]